In [0]:
import pandas as pd
import numpy as np
import os
import glob

import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import explained_variance_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

In [0]:
dataset_df = pd.read_csv("/content/drive/My Drive/Crop-Yield-Prediction/apy.csv")


In [0]:
#print(dataset_df.Season.unique())
#print(dataset_df.Crop.unique())
#print(dataset_df.State_Name.unique())
states, season, crop = [], [], []
for i in (dataset_df.State_Name.unique()):
  states.extend(i.split('\' '))
for j in dataset_df.Season.unique():
  season.extend(j.split('\' '))
for k in dataset_df.Crop.unique():
  crop.extend(k.split('\' '))
label_dict = {'Season':season, 'Crop':crop, 'State':states}

In [0]:

labels = pd.DataFrame(season)
labels.to_csv('/content/drive/My Drive/Crop-Yield-Prediction/season.csv', index = False, header=False)

In [0]:
dataset_df = dataset_df.dropna()
cr = LabelEncoder().fit(dataset_df['Crop'])
se = LabelEncoder().fit(dataset_df['Season'])
st = LabelEncoder().fit(dataset_df['State_Name'])
dataset_df['Season'] = se.transform(dataset_df['Season'])
dataset_df['Crop'] = cr.transform(dataset_df['Crop'])
dataset_df['State_Name'] = st.transform(dataset_df['State_Name'])
pickle.dump(cr, open('/content/drive/My Drive/Crop-Yield-Prediction/crop.enc','wb'))
pickle.dump(se, open('/content/drive/My Drive/Crop-Yield-Prediction/season.enc','wb'))
pickle.dump(st, open('/content/drive/My Drive/Crop-Yield-Prediction/state.enc','wb'))
dataset_df.tail()

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production
246086,32,PURULIA,2014,3,95,306.0,801.0
246087,32,PURULIA,2014,3,102,627.0,463.0
246088,32,PURULIA,2014,4,106,324.0,16250.0
246089,32,PURULIA,2014,5,95,279151.0,597899.0
246090,32,PURULIA,2014,5,102,175.0,88.0


In [0]:
Y = dataset_df['Production'].tolist()
X = dataset_df.drop(['Production', 'District_Name'], axis=1)
validation_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=validation_size)



In [0]:
model = RandomForestRegressor(max_depth=12, max_features=5, n_estimators=25)
model.fit(X_train, y_train)


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=12, max_features=5, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=25, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [0]:

y_pred=model.predict(X_test)
r1=explained_variance_score(y_test, y_pred)
print(r1)
print(model.predict([[32,2014,4,106,324]]))

0.9213468740240961
[18178.9405875]


In [0]:
file_name = '/content/drive/My Drive/Crop-Yield-Prediction/model3.h5'
pickle.dump(model, open(file_name, 'wb'))

In [0]:
model = pickle.load(open('/content/drive/My Drive/Crop-Yield-Prediction/model3.h5','rb'))
y_pred=model.predict(X_test)
    #print(explained_variance_score(y_test, y_pred))
r1=explained_variance_score(y_test, y_pred)
print(r1)
model.predict([[1,2000,4,22,720]])


0.987504230494499


array([231.00631907])

In [0]:
state = pickle.load(open('/content/drive/My Drive/Crop-Yield-Prediction/state.enc','rb'))
print(state.transform(['Andaman and Nicobar Islands']))

[0]


In [0]:
DTR = DecisionTreeRegressor()
DTR.fit(X_train, y_train)


DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [0]:
y_pred=DTR.predict(X_test)
r1=explained_variance_score(y_test, y_pred)
print(r1)
print(DTR.predict([[32,2014,4,106,324]]))

0.9013979536827172
[16250.]


In [0]:
file_name = '/content/drive/My Drive/Crop-Yield-Prediction/DTR.h5'
pickle.dump(DTR, open(file_name, 'wb'))

In [0]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression


In [0]:
KNR = LinearRegression()
KNR.fit(X_train, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
y_pred=KNR.predict(X_test)
    #print(explained_variance_score(y_test, y_pred))
r1=explained_variance_score(y_test, y_pred)
print(r1)
print(KNR.predict([[32,2014,4,106,324]]))

0.003944551604712632
[828459.27017948]


In [0]:
# scaling data:
scaler = StandardScaler()
x_trainscaled = scaler.fit_transform(X_train)
x_testscaled = scaler.transform(X_test)
x_testscaled

array([[ 0.76506251, -0.93287181, -0.91068091,  1.0134573 , -0.25623933],
       [-0.34293355,  0.88227397, -0.13349641,  0.6906035 , -0.25636647],
       [ 1.26869708,  1.48732256, -0.91068091, -0.71821309, -0.25655718],
       ...,
       [-0.24220663,  0.47890824,  1.42087259,  1.16020903, -0.25606981],
       [-0.34293355, -1.13455468,  1.42087259,  1.3363111 , -0.25615457],
       [ 0.46288177, -1.33623754, -0.91068091, -0.2779579 , -0.23464667]])

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from keras.optimizers import Optimizer
from keras import optimizers
from sklearn.metrics import mean_absolute_error

In [0]:
sgd = optimizers.SGD(lr=0.1, clipnorm = 1, nesterov = True)
ANN = Sequential()
ANN.add(Dense(256, input_dim = 5, activation='relu'))
ANN.add(Dropout(0.2))
ANN.add(Dense(128, activation='relu'))
ANN.add(Dense(1, activation='relu'))
ANN.compile(loss='mean_absolute_error', optimizer='nadam')

In [0]:
ANN.fit(X_train, y_train, epochs=50, batch_size=50) 

Epoch 1/50
169652/169652 [==============================] - 8s 49us/step - loss: 1996200.4574
Epoch 2/50
169652/169652 [==============================] - 8s 47us/step - loss: 602529.7802
Epoch 3/50
169652/169652 [==============================] - 8s 45us/step - loss: 602398.9395
Epoch 4/50
169652/169652 [==============================] - 8s 46us/step - loss: 602397.6430
Epoch 5/50
169652/169652 [==============================] - 8s 48us/step - loss: 602407.1329
Epoch 6/50
169652/169652 [==============================] - 8s 47us/step - loss: 602397.6455
Epoch 7/50
169652/169652 [==============================] - 8s 45us/step - loss: 602397.6396
Epoch 8/50
169652/169652 [==============================] - 8s 47us/step - loss: 602397.6432
Epoch 9/50
169652/169652 [==============================] - 8s 45us/step - loss: 602397.6378
Epoch 10/50
169652/169652 [==============================] - 8s 46us/step - loss: 602397.6431
Epoch 11/50
169652/169652 [==============================] - 8s 49us

In [0]:
import numpy as np
z = np.array([[32,2014,4,12,3243]])
ypredkeras = ANN.predict(x_testscaled)
mean_absolute_error(y_test, ypredkeras)
ANN.predict(z)

array([[0.]], dtype=float32)